# Cat Vs Dog using Keras

In [100]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


In [101]:
# dimensions of our images.
img_width, img_height = 224, 224

# Dataset Loading and Parameters

In [102]:
train_data_dir = 'cats-and-dogs/train'
validation_data_dir = 'cats-and-dogs/test'
nb_train_samples = 40
nb_validation_samples = 10
epochs = 2
batch_size = 4

In [103]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# Model Building

In [104]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [105]:
model.compile(loss='binary_crossentropy', # or categorical_crossentropy
              optimizer='rmsprop',# or adagrad
              metrics=['accuracy'])

In [106]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [107]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [108]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

print(train_generator.class_indices)

Found 40 images belonging to 2 classes.
{'cat': 0, 'dog': 1}


In [109]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 10 images belonging to 2 classes.


# Model Training

In [110]:
import scipy.ndimage
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples ,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples )

Epoch 1/2


C:\Users\Pranesh\AppData\Local\Temp\ipykernel_20560\2579258410.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


40/40 [==============================] - 2s 27ms/step - loss: 0.8849 - accuracy: 0.6500 - val_loss: 0.7127 - val_accuracy: 0.5000


In [111]:
model.save('model.h5')

In [112]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
from os import listdir
from os.path import isfile, join

In [113]:
# dimensions of our images
img_width, img_height = 224, 224

In [114]:
# load the model we saved
#model = load_model('model.h5')
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Prediction

In [115]:
mypath = "cats-and-dogs/valid/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
# predicting images
dog_counter = 0 
cat_counter  = 0
for file in onlyfiles:
    img = image.load_img(mypath+file, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    classes = classes[0][0]
    
    if classes == 0: 
        print(file + ": " + 'cat')
        cat_counter += 1
    else:
        print(file + ": " + 'dog')
        dog_counter += 1
print("Total Dogs :",dog_counter)
print("Total Cats :",cat_counter)

['cat.57.jpg', 'cat.58.jpg', 'cat.60.jpg', 'cat.70.jpg', 'cat.76.jpg', 'cat.78.jpg', 'cat.93.jpg', 'cat.94.jpg', 'dog.492.jpg', 'dog.493.jpg', 'dog.494.jpg', 'dog.496.jpg', 'dog.514.jpg', 'dog.531.jpg', 'dog.614.jpg', 'dog.618.jpg']
1/1 [==============================] - 0s 69ms/step


cat.57.jpg: dog
1/1 [==============================] - 0s 33ms/step
cat.58.jpg: dog
1/1 [==============================] - 0s 26ms/step
cat.60.jpg: cat
1/1 [==============================] - 0s 23ms/step
cat.70.jpg: dog
1/1 [==============================] - 0s 22ms/step
cat.76.jpg: dog
1/1 [==============================] - 0s 23ms/step
cat.78.jpg: cat
1/1 [==============================] - 0s 23ms/step
cat.93.jpg: dog
1/1 [==============================] - 0s 22ms/step
cat.94.jpg: cat
1/1 [==============================] - 0s 21ms/step
dog.492.jpg: dog
1/1 [==============================] - 0s 22ms/step
dog.493.jpg: dog
1/1 [==============================] - 0s 22ms/step
dog.494.jpg: dog
1/1 [==============================] - 0s 23ms/step
dog.496.jpg: dog
1/1 [==============================] - 0s 23ms/step
dog.514.jpg: dog
1/1 [==============================] - 0s 26ms/step
dog.531.jpg: cat
1/1 [==============================] - 0s 24ms/step
dog.614.jpg: dog
1/1 [====================